# H1st Graph Guide

H1st Graph は機械学習および人のエキスパートモデルを組み込むことができる一種のフローチャートです。

この単純なGraphの例は、入力データの偶数に対してHelloと出力します。HelloPrinterはActionノードを継承したノードです。

In [1]:
from h1st.h1flow.h1flow import Graph
from h1st.h1flow.h1step import Action, Decision, NoOp
from rule_based_model import SimpleRuleBasedModel

class HelloPrinter(Action):
    """Print hello to the inputs value"""

    def call(self, command, inputs):
        # H1stは既に条件フィルタリングを済ませているので、
        # ここでは単にprintするだけで良い。
        for d in inputs["predictions"]:
            print("Hello world {}!".format(d["value"]))

2023-02-17 10:09:46.723407: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-17 10:09:46.822035: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-17 10:09:46.822054: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-02-17 10:09:47.381567: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

Graphはadd()関数でノードを順次追加することで作成します。

最初のブランチはDecision()であり、これはRuleBasedModelの予測に基づいて入力を'yes'か'no'にリダイレクトします。

In terms of data flow, the RuleBasedModel node produces a dict of which is then used by `h1.Decision` to redirect the data stream by looking at the `result_field=predictions` dict key.

H1st graph by default operates in batch mode, meaning that `h1.Decision` looks at `{"predictions": [{"prediciton": True, ...}, {"prediction": False, }]}` and redirect True/False decision points to the to the right `yes/no` branch as a list.

In [2]:
def create_graph():
    """Create a graph which prints hello for each even number x in the input stream,
    using a conditional RuleBasedModel node and a HelloPrinter h1.Action."""
    graph = Graph()
    graph.start() \
        .add(Decision(SimpleRuleBasedModel(), result_field="predictions")) \
        .add(yes=HelloPrinter(), no=NoOp())
    graph.end()
    return graph

In [3]:
generated_graph = create_graph()
results = generated_graph.predict({"values": range(6)})

Hello world 0!
Hello world 2!
Hello world 4!
